In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm

In [3]:
balance_scale = pd.read_csv('balance-scale.data', header=None)
balance_scale.head()

,0,1,2,3,4
0,B,1,1,1,1
1,R,1,1,1,2
2,R,1,1,1,3
3,R,1,1,1,4
4,R,1,1,1,5


# a)

In [4]:
def naive_bayes(X_teste, classes_probs, medias_classes, stds_classes):
    y_hat = []
    classes = classes_probs.index
    for amostra in X_teste:
        pamostra = list()
        for c in classes:
            medias = medias_classes.loc[c]
            stds = stds_classes.loc[c]

            probs_variaveis = [norm.pdf(x, medias[i+1], stds[i+1]) for i, x in enumerate(amostra)]
            prob = np.prod(probs_variaveis)*classes_probs[c]
            pamostra.append(prob)
        
        idx_classe = np.argmax(pamostra)
        y_hat.append(classes[idx_classe])
    return y_hat

In [5]:
def acuracia(y_real, y_pred):
  comparacoes = y_real==y_pred
  acc = (sum(comparacoes)/len(comparacoes))*100
  return acc

def selecao_treino_teste(dataset, prct =0.5):
  """
  Divide a base em treino e teste

  Params:
    - dataset: dataframe do pandas
    - prct: porcentagem da base de treino a ser selecionada
  return:
    - df_treino
    - df_teste
  """
  indices = dataset.index
  size = int(prct*len(dataset))
  # escolhe os indices de treino aleatoriamente
  idx_treino = np.random.choice(indices,size=size, replace=False)
  # obtem os indices de teste
  idx_teste = list(set(indices).difference(idx_treino))
  
  # selecao dos dataframes de treino e teste
  df_treino = dataset.iloc[idx_treino]
  df_teste = dataset.iloc[idx_teste]
  
  return df_treino, df_teste

def exp_naive_gauss(n_iter):
  acuracias = []
  for i in range(n_iter):
    print ("Teste ", i)
    df_treino, df_teste = selecao_treino_teste(balance_scale, prct=.75)
    # calcula as medias e desvios padrão
    medias_classes = df_treino.groupby(0).mean()
    stds_classes = df_treino.groupby(0).std()
    # calcula as probabilidades de cada classe
    classes_probs = df_treino[0].value_counts()/len(df_treino[0])
    
    X_teste, y_teste = df_teste[[1,2,3,4]].values, df_teste[0].values
    y_pred = naive_bayes(X_teste, classes_probs, medias_classes, stds_classes)
    
    acc = acuracia(y_teste, y_pred)
    acuracias.append(acc)
    
  print ("Acurácia média: {:.4f} Desvio padrão Médio: {:.4f}".format(np.mean(acuracias), np.std(acuracias)))

In [7]:
exp_naive_gauss(10)

Teste  0
Teste  1
Teste  2
Teste  3
Teste  4
Teste  5
Teste  6
Teste  7
Teste  8
Teste  9
Acurácia média: 90.6369 Desvio padrão Médio: 2.2438


# b)

In [8]:
def discretizar_atributos(base):
    dic_classe_prob = {}
    colunas = [1,2,3,4]
    intervalos = [1,2,3,4,5]
    classes = dict(base[0].value_counts())
    for classe in classes:
        dic_classe_prob.setdefault(classe, {})
        classe_c = base[base[0]==classe]
        for c in colunas:
            dic_classe_prob[classe].setdefault(c, {})
            for intv in intervalos:
                t = (intv, intv+1)
                valores = classe_c[(classe_c[c]>=t[0]) & (classe_c[c] < t[1])]
                dic_classe_prob[classe][c][t] = len(valores)
    return dic_classe_prob

def classify_discret_prop(dic_classe_prob, X_teste, X_treino, laplace=False):
    y_pred = []
    intervalos = [1,2,3,4,5]
    for amostra in X_teste.values:
        colunas = [1,2,3,4]
        probs_amostra = {}
        for c in dic_classe_prob:
            probs = []
            for cl in colunas:
                valor = amostra[cl]
                for intv in intervalos:
                    t = (intv, intv+1)            
                    if (valor>=t[0] and valor<t[1]):
                        if laplace:
                            pcl = (dic_classe_prob[c][cl][(t[0],t[1])]+1)/(X_treino.shape[0]+5)
                        else:
                            pcl = dic_classe_prob[c][cl][(t[0],t[1])]/X_treino.shape[0]
                probs.append(pcl)
            probs_amostra[c] = sum(probs)
        
        pred = sorted(probs_amostra.items(), key=lambda item: item[1])[-1][0]
        y_pred.append(pred)
    
    return y_pred

def exp_discret_naive(nexp, laplace=False):
    accuracies = []
    for i in range(nexp):
        df_treino, df_teste = selecao_treino_teste(balance_scale, prct=.75)
        dic_classe_prob = discretizar_atributos(df_treino)
        y_pred = classify_discret_prop(dic_classe_prob, df_teste, df_treino, laplace=laplace)
        acc = acuracia(df_teste[0].values, y_pred)
        accuracies.append(acc)
    m_ = np.mean(accuracies)
    d_ = np.std(accuracies)
    print ("Acurácia Média {:.4f}, Desvio Padrão Médio {:.4f}".format(m_, d_))

In [9]:
exp_discret_naive(10)

Acurácia Média 90.0637, Desvio Padrão Médio 1.4580


# c)

In [10]:
exp_discret_naive(10, laplace=True)

Acurácia Média 90.8280, Desvio Padrão Médio 1.9151
